# Day 19

https://adventofcode.com/2020/day/19

## Part 1

After several attempts, ultimately I reckon I cannot solve it withour using regexp. so I need to learn the basics...

In [412]:
# Here's how I can use a rule as a regexp, using the OR sign and parentheses...
import re
bool(re.fullmatch('a(b|a)','aa'))

True

In [453]:
def getRules(p):
    
    # store rules in dictionary with rule number as key (as a string)
    rules = {}
    for l in p:
        i, r = l.split(": ")
        rules[i] = r.replace('"','')
    
    # add parentesys (and space to that later split will work) if there's an OR symbol in rule.
    for r in rules.keys():
        if "|" in rules[r]:
            rules[r] = "( "+rules[r]+" )"

    return rules
        

def solveRules(rules,debug=False):

    # dictionay of solved rules
    rules_solved = {}

    while '0' not in rules_solved.keys():

        for k in rules.keys():
            if k in rules_solved.keys():
                continue

            r = rules[k]        
            if not re.findall("[0-9]",rules[k]): # rule is solved, store it
                rules_solved[k] = r
            else: # otherwise see whether I can replace rule number with corresponding rule
                l = r.split(" ")
                for i in range(len(l)):
                    j = l[i]
                    if j in rules_solved.keys():
                        l[i] = rules_solved[j] 
                nr = "" # reform rule string after replacements
                for j in l:
                    nr += j+" "
                nr = nr[:-1] # remove last white space
                rules[k] = nr
        if debug:     
            print(len(rules),len(rules_solved))
            print(rules['0'].replace(" ",""))

    return rules_solved

In [520]:
#filename = "data/day19test1.txt"
filename = "data/input19.txt"

with open(filename) as f:
    p = [b.split("\n") for b in f.read().split("\n\n")]

messages = p[1][:-1]
rules = getRules(p[0])
rules_solved = solveRules(rules)

rule0 = rules_solved['0'].replace(" ","") # remove all white spaces from rule 0
#print(rule0)

import re
match = sum( [ bool(re.fullmatch(rule0, m)) for m in messages ] )
print('Solution Part 1 =',match)

Solution Part 1 = 126


## Part 2

In [521]:
filename = "data/day19test2.txt"

with open(filename) as f:
    p = [b.split("\n") for b in f.read().split("\n\n")]

messages = p[1][:-1]
rules = getRules(p[0])
rules_solved = solveRules(rules)

rule0 = rules_solved['0'].replace(" ","") # remove all white spaces from rule 0
#print(rule0)

match = sum( [ bool(re.fullmatch(rule0, m)) for m in messages ] )
print('Solution Test 2 =',match)

Solution Test 2 = 3


### Notes

* There seem to be only rule 0 to depend on rule 8 and rule 11, so loop only happens there
* I can begin toremove rules 0, 8 and 11 from `rules`, and attempt to solve all others since they should not be affected by loop. I can probably use the same solving routine than in Part 1, but with a different ending condition

In [604]:
filename = "data/day19test2.txt"

with open(filename) as f:
    p = [b.split("\n") for b in f.read().split("\n\n")]

messages = p[1][:-1]
rules = getRules(p[0])

# New rules
r0 = rules['0']
r8 = "(42 | 42 8)" # 8
r11 = "(42 31 | 42 11 31)" # 11

print(r0,r8,r11)

rules.pop('0')
rules.pop('8')
rules.pop('11')

#rules

8 11 (42 | 42 8) (42 31 | 42 11 31)


'42 31'

In [605]:
def solveRules2(rules,debug=False):

    # dictionay of solved rules
    rules_solved = {}

    while len(rules) !=len(rules_solved): # change condition wrt to Part 1, now looking to solve all rules
        
        for k in rules.keys():
            if k in rules_solved.keys():
                continue

            r = rules[k]        
            if not re.findall("[0-9]",rules[k]): # rule is solved, store it
                rules_solved[k] = r
            else: # otherwise see whether I can replace rule number with corresponding rule
                l = r.split(" ")
                for i in range(len(l)):
                    j = l[i]
                    if j in rules_solved.keys():
                        l[i] = rules_solved[j] 
                nr = "" # reform rule string after replacements
                for j in l:
                    nr += j+" "
                nr = nr[:-1] # remove last white space
                rules[k] = nr
    
        #print(len(rules),len(rules_solved))

    return rules_solved

In [606]:
rules_solved = solveRules2(rules)
print("42:", rules_solved['42'])
print("31:", rules_solved['31'])

42: ( ( b ( a ( b b | a b ) | b ( a | b ) ( a | b ) ) | a ( b b b | a ( b b | a ( a | b ) ) ) ) b | ( ( ( a a | a b ) a | b b b ) b | ( ( a | b ) a | b b ) a a ) a )
31: ( b ( b ( a b a | b a a ) | a ( b ( a b | ( a | b ) a ) | a ( b a | a b ) ) ) | a ( b ( ( a b | ( a | b ) a ) b | ( ( a | b ) a | b b ) a ) | a ( b a b | ( b a | b b ) a ) ) )


From the puzzle test:

> it might help to start by looking at which rules always match the same set of values and how those rules (especially rules 42 and 31) are used by the new versions of rules 8 and 11.

Now I have solved rules 42 and 31, all the ingredients needed to build rules 8 and 11 and therefore rule 0.

`0: 8 11`

`8: 42 | 42 8`

`42 31 | 42 11 31`

Rule 8 is itself or itself repeated...

Rule 11 is `42+31` or `42+42+31+31` or `42+42+42+32+31+31` and so on... 

(after some studyin of regexp...)

For Rule 8 can use the RE `+` symbol on Rule 42.

For Rule 11 I can use `{m}` to repeate rules 42 and 31 `m` times.

Note that for Rule 8 there's nothing more to be done since `+` takes care of everything, while for Rule 11 I need to make a loop over `m` and count how many expressions match as a function of `m`, hoping that at some point the number stabilizes...

In [608]:
# Rule 8 with +
# ==> + Causes the resulting RE to match 1 or more repetitions of the preceding RE
rules_solved['8'] = rules_solved['42']+ '+'
    
# Rule 11 with variable {n}, to be replaced during counting by numerical values
# ==> {m} Specifies that exactly m copies of the previous RE should be matched; 
#     fewer matches cause the entire RE not to match. 
rules_solved['11'] = rules_solved['42'] + r'{m}' + rules_solved['31'] + r'{m}'

# Rule 0
rules_solved['0'] = rules_solved['8']+rules_solved['11']

# Being with 1 repetition to initialize counter
n = 1
rule0 = rules_solved['0'].replace(" ","").replace('m',str(n)) 
count_all = sum([bool(re.fullmatch(rule0, m)) for m in messages])
count_pre = 0

# Increase RE repetitions untile number of matched expression saturates to asymptotic value
while count_pre != count_all:
    count_pre = count_all
    n += 1
    rule0 = rules_solved['0'].replace(" ","").replace('m',str(n)) # increase regex repetition
    count_n = sum([bool(re.fullmatch(rule0, m)) for m in messages])
    count_all += count_n
    print(n,count_n,count_all,count_pre)

print('Solution Test 2 after replacing =',count_all)

2 3 9 6
3 2 11 9
4 1 12 11
5 0 12 12
Solution Test 2 after replacing = 12


In [609]:
filename = "data/input19.txt"

with open(filename) as f:
    p = [b.split("\n") for b in f.read().split("\n\n")]

messages = p[1][:-1]
rules = getRules(p[0])
rules.pop('0')
rules.pop('8')
rules.pop('11')
rules_solved = solveRules2(rules)

rules_solved['8'] = rules_solved['42']+ '+'
rules_solved['11'] = rules_solved['42'] + r'{m}' + rules_solved['31'] + r'{m}'
rules_solved['0'] = rules_solved['8']+rules_solved['11']

n = 1
rule0 = rules_solved['0'].replace(" ","").replace('m',str(n)) 
count_all = sum([bool(re.fullmatch(rule0, m)) for m in messages])
count_pre = 0

while count_pre != count_all:
    count_pre = count_all
    n += 1
    rule0 = rules_solved['0'].replace(" ","").replace('m',str(n))
    count_n = sum([bool(re.fullmatch(rule0, m)) for m in messages])
    count_all += count_n

print('Solution Part 2 after replacing =',count_all)

Solution Part 2 after replacing = 282
